# Machine Learning using Spark on Digital Music Service Data

## Table of Contents

I. [Part 1: Load and Clean Dataset](#part1)<br>
II. [Part 2: Exploratory Data Analysis](#part2)<br>
III. [Part 3: Feature Engineering](#part3)<br>
IV. [Part 4: Modeling](#part4)<br>
$\;\;\;\;\;\;$[4.1 ](#part4.1)<br>
$\;\;\;\;\;\;$[4.2 ](#part4.2)<br>
V. [Part 5: Conclusion](#part5)<br>

In [1]:
import findspark

findspark.init()

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat,count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [3]:
import re
import datetime

## <a class="anchor" id="part1"> Part 1: Load and Clean Dataset </a>

In this workspace, the dataset file is medium_sparkify_event_data.json. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids.

In [4]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

In [5]:
sparkify_data = 'medium-sparkify-event-data.json'

In [6]:
df = spark.read.json(sparkify_data)
df.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [7]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
df.select('page').dropDuplicates().sort('page').show(30)

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|               Login|
|              Logout|
|            NextSong|
|            Register|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
| Submit Registration|
|      Submit Upgrade|
|         Thumbs Down|
|           Thumbs Up|
|             Upgrade|
+--------------------+



In [9]:
df.select('userId').dropDuplicates().sort('userId').show(3)

+------+
|userId|
+------+
|      |
|    10|
|   100|
+------+
only showing top 3 rows



### Drop Rows with Missing Values

As you'll see, it turns out there are no missing values in the userID or session columns. But there are userID values that are empty strings.

In [10]:
df.count()

543705

In [8]:
df_valid = df.dropna(how = "any", subset = ["userId", "sessionId"])

In [9]:
df_valid.count()

543705

In [10]:
df_valid = df_valid.filter(df_valid["userId"] != "") 

In [11]:
df_valid.count()

528005

Change 'ts' to datetime

In [12]:
get_time = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%Y-%m-%d %H:%M:%S"))

# Apply function to column 'ts'
df_valid = df_valid.withColumn('time', get_time(df_valid.ts))

In [13]:
df_valid.head()

Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='293', time='2018-09-30 19:00:11')

## <a class="anchor" id="part2"> Part 2: Exploratory Data Analysis </a>

When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark.

In [14]:
#Create a temporary view of SQL table from dataframe "df_valid"

df_valid.createOrReplaceTempView("df_cleaned")

## <a class="anchor" id="part3"> Part 3: Feature Engineering </a>

Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.

* Write a script to extract the necessary features from the smaller subset of data
* Ensure that your script is scalable, using the best practices discussed in Lesson 3
* Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [28]:
df_valid.filter("page = 'Cancellation Confirmation'").count()

99

In [16]:
flag_downgrade_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

df_valid = df_valid.withColumn("Churn", flag_downgrade_event("page"))

## <a class="anchor" id="part4"> Part 4: Modeling </a>

Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [17]:
# TODO: write your code for this step
train, validation = df_valid.randomSplit([0.9, 0.1], seed=42)

### Step 2. Build Pipeline

In [20]:
#build pipeline
Gender_indexer = StringIndexer(inputCol="gender", outputCol='Gender_Index')
User_indexer = StringIndexer(inputCol="userAgent", outputCol='User_Index')
Page_indexer = StringIndexer(inputCol="page", outputCol='Page_Index')
Gender_encoder = OneHotEncoder(inputCol='Gender_Index', outputCol='Gender_Vec')
User_encoder = OneHotEncoder(inputCol='User_Index', outputCol='User_Vec')
Page_encoder = OneHotEncoder(inputCol='Page_Index', outputCol='Page_Vec')
assembler = VectorAssembler(inputCols=["Gender_Vec", "User_Vec", "Page_Vec", "status"], outputCol="features")
indexer = StringIndexer(inputCol="Churn", outputCol="label")

In [21]:
lr =  LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0)

pipeline = Pipeline(stages=[Gender_indexer, User_indexer, Page_indexer, Gender_encoder, User_encoder, Page_encoder, assembler, indexer, lr])

### Step 3. Tune Model
On the first 90% of the data let's find the most accurate logistic regression model using 3-fold cross-validation with the following parameter grid:

- CountVectorizer vocabulary size: `[1000, 5000]`
- LogisticRegression regularization parameter: `[0.0, 0.1]`
- LogisticRegression max Iteration number: `[10]`

In [23]:
# TODO: write your code for this step
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam,[0.0, 0.1]) \
    .build()


crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)

### Step 4: Compute Accuracy of Best Model

In [24]:
# TODO: write your code for this step
cvModel_q1 = crossval.fit(train)

In [25]:
cvModel_q1.avgMetrics

[1.0, 0.9997126027454916]

In [26]:
results = cvModel_q1.transform(validation)

In [27]:
print(results.filter(results.label == results.prediction).count())
print(results.count())

53100
53100


In [14]:
3919/10062

0.3894851918107732

## <a class="anchor" id="part5"> Part 5: Conclusion </a>